# Lecture 23: Sharing and collaboration

IN PROGRESS

## Hosted notebooks (Google Colab)

## Version control and collaboration (Git/GitHub)

## Integrated development environments (IDEs; Visual Studio Code)

## Command line

- UNIX vs. Windows
- Git BASH